# Importando bibliotecas

In [ ]:
import logging
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd
from stable_baselines3 import DQN
from stable_baselines3.dqn.policies import MlpPolicy
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import time
from datetime import datetime
from torch import nn  # Import the neural network module from PyTorch
from sklearn.model_selection import train_test_split
from stable_baselines3 import DQN, PPO
from stable_baselines3.dqn.policies import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from sklearn.utils import resample
from tensorflow.keras.models import load_model

import sys
from stable_baselines3 import A2C

import os
sys.path.append(os.path.join('..'))
from classes._exploration import exploration
from classes._Env3WGym import Env3WGym
from classes._LoadInstances import LoadInstances
from classes._Agent import Agent
from classes._Supervised import Supervised
from classes._ValidationModel import ValidationModel
from classes._exploration import exploration

# Validação dos dados simulados

In [ ]:
if __name__ == '__main__':

    events_names = {
        # 0: 'Normal',
        1: 'Abrupt Increase of BSW',
        # 2: 'Spurious Closure of DHSV',
        # 3: 'Severe Slugging',
        # 4: 'Flow Instability',
        # 5: 'Rapid Productivity Loss',
        # 6: 'Quick Restriction in PCK',
        # 7: 'Scaling in PCK',
        # 8: 'Hydrate in Production Line'
    }

    event_name = [value for key, value in events_names.items() if key != 0][0]
    
    models = ['DQN'] # 'DQN' , 'PPO', 'A2C', 'RNA'
    type_instance= 'simulated' # real, simulated
    path_dataset = '..\\..\\..\\dataset'
    instances = LoadInstances(path_dataset)
            
    logging.info(f'Iniciando carregamento do dataset')
    dataset, _ = instances.load_instance_with_numpy(events_names, type_instance=type_instance)    
    logging.info(f'Fim carregamento do dataset')
    
    logging.info(f'Iniciando divisão do dataset em treino e teste')
        
    # Definindo a porcentagem para divisão entre treino e teste
    train_percentage = 0.8  # 80% para treino

    dataset_train_scaled, dataset_test_scaled, dataset_validation_scaled = instances.data_preparation(dataset, train_percentage)

    env_train = Env3WGym(dataset_train_scaled, n_envs = 5)
    env_test = Env3WGym(dataset_test_scaled, n_envs = 1)

    envs_train = env_train.envs_random()
    envs_test = env_test.envs_random()

    path_tensorboard = f'..\\models\\{event_name}-{type_instance}'   
    if not os.path.exists(path_tensorboard):
                os.makedirs(path_tensorboard)

    agente = Agent(path_tensorboard, envs_train, envs_test, TIMESTEPS = 1000000)
    
    for model_type in models:
        for event_name in [value for key, value in events_names.items() if key != 0]:

            directory = f'..\\..\\logs\\{event_name}-{type_instance}'               
            path_model = f'..\\models\\{event_name}-{type_instance}\\{model_type}'  

            if not os.path.exists(directory):
                os.makedirs(directory)
            
            if not os.path.exists(path_model):
                os.makedirs(path_model)   
                    
            current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            log_filename = f'{directory}\\{current_time}_{event_name}-{type_instance}_{model_type}-log.txt'
            # Configuração do Logging
            logging.basicConfig(filename=log_filename, filemode='w', level=logging.INFO, format='[%(levelname)s]\t%(asctime)s - %(message)s', datefmt='%d/%m/%Y %I:%M:%S %p', force=True, encoding='utf-8')
          

            logging.info(f'Iniciando a execução do algoritmo {model_type}-{type_instance} para o evento {event_name} com timesteps {agente.TIMESTEPS}')
            
            if model_type == 'DQN':
                
                try:
                    logging.info(f'Iniciando treinamento do algoritmo DQN')    
                    start_time = time.time()                
                    model_agent, replaydir = agente.env3W_dqn(path_save=path_model)  
                    print(f"Tempo de Treinamento DQN: {round(time.time() - start_time, 2)}s")
                    logging.info(f"Tempo de Treinamento DQN: {round(time.time() - start_time, 2)}s")
                    logging.info(f'Fim treinamento do algoritmo DQN')

                    try:
                        logging.info(f'Iniciando avaliação do algoritmo DQN conjunto de teste')
                        accuracy = agente.env3W_dqn_eval(model = model_agent, path_save=path_model)
                        print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando DQN')
                        logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando DQN')
                        logging.info(f'Fim avaliação  do algoritmo DQN conjunto de teste')
                    except Exception as e:
                        logging.error(f'Erro ao avaliar o modelo DQN: {e}')

                except Exception as e:
                    logging.error(f'Erro ao treinar o modelo DQN: {e}')
                
            elif model_type == 'PPO':      
                
               
                logging.info(f'Iniciando treinamento do algoritmo PPO') 
                start_time = time.time()
                #agente = Agent(path_model)
                model_agent = agente.env3W_ppo(path_save=path_model)  
                print(f"Tempo de Treinamento PPO: {round(time.time() - start_time, 2)}s")
                logging.info(f"Tempo de Treinamento PPO: {round(time.time() - start_time, 2)}s")
                logging.info(f'Fim treinamento do algoritmo PPO')
            
                
                logging.info(f'Iniciando avaliação do algoritmo PPO conjunto de teste')
                accuracy = agente.env3W_ppo_eval(model = model_agent, path_save=path_model)
                print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando PPO')
                logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando PPO')
                logging.info(f'Fim avaliação  do algoritmo PPO conjunto de teste')
               
            elif model_type == 'A2C': 

                try:     
                    logging.info(f'Iniciando treinamento do algoritmo A2C') 
                    start_time = time.time()
                    #agente = Agent(path_model)
                    model_agent = agente.env3W_a2c(path_save=path_model)  
                    print(f"Tempo de Treinamento A2C: {round(time.time() - start_time, 2)}s")
                    logging.info(f"Tempo de Treinamento A2C: {round(time.time() - start_time, 2)}s")
                    logging.info(f'Fim treinamento do algoritmo A2C')
                
                    try:
                        logging.info(f'Iniciando avaliação do algoritmo A2C conjunto de teste')
                        accuracy = agente.env3W_a2c_eval(model = model_agent, path_save=path_model)
                        print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando A2C')
                        logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando A2C')
                        logging.info(f'Fim avaliação  do algoritmo A2C conjunto de teste')
                    except Exception as e:
                        logging.error(f'Erro ao avaliar o modelo A2C: {e}')

                except Exception as e:
                    logging.error(f'Erro ao treinar o modelo A2C: {e}')
            
            elif model_type == 'RNA':
                try:
                    logging.info(f'Iniciando treinamento do modelo RNA') 
                    dataset_train_scaled[:, -1] = np.where(dataset_train_scaled[:, -1] == 101, 1, dataset_train_scaled[:, -1])
                    dataset_test_scaled[:, -1] = np.where(dataset_test_scaled[:, -1] == 101, 1, dataset_test_scaled[:, -1])
                    supervised = Supervised(path_model, dataset_train_scaled, dataset_test_scaled)
                    start_time = time.time()
                    model_agent = supervised.keras_train()  
                    print(f"Tempo de Treinamento RNA: {round(time.time() - start_time, 2)}s")
                    logging.info(f"Tempo de Treinamento RNA: {round(time.time() - start_time, 2)}s")
                    logging.info(f'Fim treinamento do modelo RNA')

                    try:
                        logging.info(f'Iniciando avaliação do modelo RNA conjunto de teste')
                        accuracy = supervised.keras_evaluate(model_agent)
                        print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando RNA')
                        logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando RNA')
                        logging.info(f'Fim avaliação  do modelo RNA conjunto de teste')
                    except Exception as e:
                        logging.error(f'Erro ao treinar o modelo RNA: {e}')
                except Exception as e:
                    logging.error(f'Erro ao treinar o modelo RNA: {e}')

            #logging.info(f'Iniciando a validação do modelo {model_type}') 
            #validation = ValidationModel(model_type, event_name)
            #validation.validation_model(accuracy, dataset_validation_scaled, model_agent)

            logging.info(f'Concluído a execução do algoritmo {model_type}-{type_instance} para o evento {event_name}')
            # Para fechar explicitamente o arquivo de log, obtenha todos os handlers do root logger e feche-os
            for handler in logging.root.handlers[:]:
                handler.close()
                logging.root.removeHandler(handler)
   

# Geração de gráficos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dados extraídos das imagens
modelos = ['A2C', 'DQN', 'PPO']  # , 'RNA'
valores_epoch_100000 = [84.23, 96.56, 84.85]  # , 0.963
valores_epoch_1000000 = [86.98, 98.23, 96.58]  # , 0.963

# Largura das barras
bar_width = 0.35

# Configurando a posição das barras
r1 = np.arange(len(valores_epoch_100000))
r2 = [x + bar_width for x in r1]

# Criando um gráfico de barras agrupadas
plt.figure(figsize=(10, 6))

# Desenhando as barras
plt.bar(r1, valores_epoch_100000, color='#FF3399', width=bar_width, label='100k passos')
plt.bar(r2, valores_epoch_1000000, color='#F79646', width=bar_width, label='1M passos')

# Adicionando rótulos, título e personalizando eixos
plt.xlabel('Modelos', fontweight='bold')
plt.xticks([r + bar_width/2 for r in range(len(valores_epoch_100000))], modelos)
plt.ylabel('Acurácia %')
plt.ylim(80, 100)
#plt.title('Acurácia dos Modelos por Número de Passos')

# Adicionando a grade e a legenda
plt.grid(True)
plt.legend()


# Show the graphs
plt.tight_layout()
plt.grid(True)
plt.savefig('..\\..\\img\\grafico_acuracia.png', dpi=300)
plt.show()


# Validação com dados reais

In [ ]:
if __name__ == '__main__':

    events_names = {
        # 0: 'Normal',
        1: 'Abrupt Increase of BSW',
        # 2: 'Spurious Closure of DHSV',
        # 3: 'Severe Slugging',
        # 4: 'Flow Instability',
        # 5: 'Rapid Productivity Loss',
        # 6: 'Quick Restriction in PCK',
        # 7: 'Scaling in PCK',
        # 8: 'Hydrate in Production Line'
    }

    event_name = [value for key, value in events_names.items() if key != 0][0]
    
    models = ['DQN'] # 'DQN' 
    type_instance= 'real' # real, simulated
    path_dataset = '..\\..\\..\\dataset'
    instances = LoadInstances(path_dataset)
    model_type = 'DQN'
            
    logging.info(f'Iniciando carregamento do dataset')
    dataset, _ = instances.load_instance_with_numpy(events_names, type_instance=type_instance)    
    logging.info(f'Fim carregamento do dataset')    
    
    logging.info(f'Iniciando divisão do dataset em treino e teste')        
    # Definindo a porcentagem para divisão entre treino e teste
    train_percentage = 0.8  # 80% para treino
    dataset_train_scaled, dataset_test_scaled, dataset_validation_scaled = instances.data_preparation(dataset, train_percentage)
   
    env_train = Env3WGym(dataset_train_scaled, n_envs = 5)
    env_test = Env3WGym(dataset_test_scaled, n_envs = 1)

    envs_train = env_train.envs_random()
    envs_test = env_test.envs_random()

    env_test_cl = Env3WGym(dataset_test_scaled, n_envs = 1)
    envs_cl = env_test_cl.envs_random()

    directory = f'..\\..\\logs\\{event_name}-{type_instance}-CL'
    path_dataset = '..\\..\\..\\dataset'   
    path_model = f'..\\models\\{event_name}-{type_instance}\\{model_type}'  

    if not os.path.exists(directory):
        os.makedirs(directory)

    if not os.path.exists(path_model):
        os.makedirs(path_model)    

    path_tensorboard = f'..\\models\\{event_name}-{type_instance}-CL'   
    if not os.path.exists(path_tensorboard):
                os.makedirs(path_tensorboard)

    agente = Agent(path_tensorboard, envs_train, envs_test, TIMESTEPS = 100000)            

            
    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_filename = f'{directory}\\{current_time}_{event_name}-{type_instance}_{model_type}-log.txt'
    # Configuração do Logging
    logging.basicConfig(filename=log_filename, filemode='w', level=logging.INFO, format='[%(levelname)s]\t%(asctime)s - %(message)s', datefmt='%d/%m/%Y %I:%M:%S %p', force=True, encoding='utf-8')
   

    if model_type == 'DQN':
        logging.info(f'Iniciando o teste do algoritmo DQN (Continual Learning)')    
        #start_time = time.time()
        #model_agent = agente.env3W_dqn(path_save=path_model)  
        #print(f"Tempo de Treinamento DQN: {round(time.time() - start_time, 2)}s")
        #logging.info(f"Tempo de Treinamento DQN: {round(time.time() - start_time, 2)}s")
        #logging.info(f'Fim treinamento do algoritmo DQN')

        #os.path.join("..", "models", "Abrupt Increase of BSW-simulated", "DQN", "dqn_save_replay_buffer")

        path_model = os.path.join("..", "models", "Abrupt Increase of BSW-simulated-100k", "DQN", "_DQN.zip")
        model_agent = DQN.load(path_model) 
        logging.info(f'Iniciando avaliação do algoritmo DQN conjunto de teste')
        accuracy = agente.env3W_dqn_eval(model = model_agent, path_save=path_model)
        print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando DQN')
        logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando DQN')
        logging.info(f'Fim avaliação  do algoritmo DQN conjunto de teste')
        
        replay_buffer_path = os.path.join("..", "models", "Abrupt Increase of BSW-simulated", "DQN", "dqn_save_replay_buffer") 
        model_agent_cl = agente.env3W_dqn_cl(model_agent, envs_cl, replaydir = replay_buffer_path)
        logging.info(f'Iniciando avaliação do algoritmo DQN conjunto de teste (Continual Learning)')
        accuracy = agente.env3W_dqn_eval(model = model_agent_cl, path_save=path_model)
        print(f'Acurácia de {accuracy * 100:.2f}% no conjunto de dados de teste usando DQN (Continual Learning)')
        logging.info(f'Acurácia de {accuracy:.5f} no conjunto de dados de teste usando DQN (Continual Learning)')
        logging.info(f'Fim avaliação  do algoritmo DQN conjunto de teste (Continual Learning)')   

    # Para fechar explicitamente o arquivo de log, obtenha todos os handlers do root logger e feche-os
    for handler in logging.root.handlers[:]:
        handler.close()
        logging.root.removeHandler(handler)
    

# Exploração dos dados

In [ ]:
'''events_names = {
    # 0: 'Normal',
    1: 'Abrupt Increase of BSW',
    # 2: 'Spurious Closure of DHSV',
    # 3: 'Severe Slugging',
    # 4: 'Flow Instability',
    # 5: 'Rapid Productivity Loss',
    # 6: 'Quick Restriction in PCK',
    # 7: 'Scaling in PCK',
    # 8: 'Hydrate in Production Line'
}

event_name = [value for key, value in events_names.items() if key != 0][0]


type_instance='simulated'

instances = LoadInstances(path_dataset)
            

dataset, _ = instances.load_instance_with_numpy(events_names, type_instance=type_instance)'''

In [ ]:


'''explora = exploration(pd.DataFrame(dataset, columns=['timestamp', 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'class']))

explora.quartiles_plot(['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP'], f'Quartis das variáveis {type_instance} do evento {event_name}')'''